In [ ]:
!pip uninstall wrds

In [12]:
!pip install --upgrade wrds

In [47]:
import wrds
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)

In [2]:
db = wrds.Connection(wrds_username='christiaank')


Loading library list...
Done


In [ ]:
db.list_libraries()

In [36]:
table_list2 = [item for item in table_list if 'ctm' in item and len(item) < 9]

table_list2

['ctm_2003',
 'ctm_2004',
 'ctm_2005',
 'ctm_2006',
 'ctm_2007',
 'ctm_2008',
 'ctm_2009',
 'ctm_2010',
 'ctm_2011',
 'ctm_2012',
 'ctm_2013',
 'ctm_2014',
 'ctm_2015',
 'ctm_2016',
 'ctm_2017',
 'ctm_2018',
 'ctm_2019',
 'ctm_2020',
 'ctm_2021',
 'ctm_2022',
 'ctm_2023']

In [27]:
# db.describe_table(library='taqmsec', table='ctm_2018')
db.describe_table(library='taqmsec', table='ctm_20180102')


Approximately 9196317945 rows in taqmsec.ctm_2018.


,name,nullable,type,comment
0,date,True,DATE,None
1,time_m,True,TIME,None
2,time_m_nano,True,SMALLINT,None
3,ex,True,VARCHAR(1),None
4,sym_root,True,TEXT,None
5,sym_suffix,True,TEXT,None
6,tr_scond,True,VARCHAR(4),None
7,size,True,INTEGER,None
8,price,True,NUMERIC,None
9,tr_stop_ind,True,VARCHAR(1),None


In [43]:
parm = {'syms': ('GME', 'AMD', 'AAPL'), 'num_shares': 100000, 'end_date': '2018-01-31'}
data = db.raw_sql('select date, time_m, ex, sym_root, size, price from taqmsec.ctm_2018 where sym_root in %(syms)s and size > %(num_shares)s and date < %(end_date)s', params=parm)
data.head()

,date,time_m,ex,sym_root,size,price
0,2018-01-05,09:30:00.130999,Q,AAPL,548618,173.51
1,2018-01-05,09:30:00.131141,Q,AAPL,548618,173.51
2,2018-01-05,15:44:44.788170,M,AAPL,233674,175.00
3,2018-01-05,16:00:00.142643,Q,AAPL,2043648,175.00
4,2018-01-05,16:00:00.142655,Q,AAPL,2043648,175.00


In [ ]:
# data = db.raw_sql('select date, time_m, ex, sym_root, size, price from taqmsec.ctm_20180102 where sym_root in %(syms)s and size > %(num_shares)s', params=parm)


In [48]:
data.tail(100)

,date,time_m,ex,sym_root,size,price
380,2018-01-09,09:30:00.754394,Q,AMD,817428,12.0000
381,2018-01-22,15:48:51.826853,D,AMD,347019,12.6200
382,2018-01-22,16:00:00.353279,Q,AMD,575684,12.6500
383,2018-01-09,15:04:26.503363,D,AMD,250000,11.8700
384,2018-01-18,13:58:20.356569,D,AMD,281200,12.5300
385,2018-01-18,14:07:44.216687,M,AMD,277500,12.4919
386,2018-01-10,12:50:59.092068,D,AMD,148940,11.9950
387,2018-01-22,16:00:00.353338,Q,AMD,575684,12.6500
388,2018-01-22,11:57:42.145110,M,GME,465000,17.5900
389,2018-01-10,14:11:23.437286,D,AMD,562500,12.0300


In [22]:
taq_data = db.get_table(library='taqmsec', table='ctm_20180501',
                        columns = ['date', 'time_m', 'ex', 'sym_root', 'size', 'price'], obs=5)


In [23]:
taq_data

,date,time_m,ex,sym_root,size,price
0,2018-05-01,09:30:01.004489,N,A,14746,65.63
1,2018-05-01,09:30:01.464259,N,A,20,65.82
2,2018-05-01,09:30:01.498533,D,A,1,65.83
3,2018-05-01,09:30:01.560510,T,A,60,65.64
4,2018-05-01,09:30:01.560533,T,A,60,65.64
